In [1]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import requests
import time

strkey=input("請輸入欲查詢的關鍵字:")
comurl=input("請輸入欲比對的公司網址或名稱:")
#===== requests get Google 搜尋資料10頁 =====
bsli=[]
for page in range(0,100,10):
    url="https://www.google.com/search?q=" +  strkey + "&start=" + str(page)
    time.sleep(1)
    print("Google搜尋資料擷取進度:{:>3}%".format(page))
    predata=requests.get(url,timeout=(5,10))
    
    bsdata=BS(predata.text,'lxml')
    bsli+=bsdata.select('#main > div > div > div.kCrYT > a') # > 與CSS參數間必需有空隔在colab中才能正確執行 
print("Google搜尋資料擷取進度:{}%".format('100'))

#===== 搜尋結果排名計算 =====
num=[]
for i in range(len(bsli)):
    #print(i+1,bsli[i].text)
    #print(bsli[i]['href'])
    if (comurl in bsli[i].text):
        num.append(i)

print("【{}】 在Google以 【{}】關鍵字搜尋結果：".format(comurl,strkey))
if len(num)!=0:
    for j in num:
        print("【{}】搜尋的名次為 {} 名.".format(bsli[j].text,j+1))
else:
    print("【{}】搜尋不在前100名內".format(comurl))
    
#===== 取前十筆有效meta name=keyword 的內容 =====
keywords=[]
count=0
for i in range(len(bsli)) :
    urlB=bsli[i].get('href')
    suburl="https://www.google.com.tw"+urlB
    #print(suburl)
    try:
        time.sleep(1)
        kwData=requests.get(suburl,timeout=(3,8))
        if 'text/html' in kwData.headers['Content-Type']:
            if "charset=big5" in kwData.text:kwData.encoding='big5'
            elif "charset=utf-8" in kwData.text:kwData.encoding='utf-8'
            elif "charset=UTF-8" in kwData.text:kwData.encoding="UTF-8"
            bsKey=BS(kwData.text,'html.parser')
            if ('keywords' in kwData.text):
                keyword=bsKey.find('meta',{'name':'keywords'})
                if keyword.get("content") !=None:
                    keywords.append(keyword.get("content"))
                    count+=1
                    print("關鍵字分析進度:{:>3}%".format(count*10))
            elif ('Keywords' in kwData.text):
                keyword=bsKey.find('meta',{'name':'Keywords'})
                if keyword.get('content')!= None:
                    keywords.append(keyword.get('content'))
                    count+=1
                    print("關鍵字分析進度:{:>3}%".format(count*10))
            if count>=10:break
    except:
        pass

#print(keywords)

#----- 整理收集到的關鍵字並建立一串列 -----
totalKeys=[]
for i in range (len(keywords)):
    tempstr=keywords[i]
    tempstr=tempstr.replace("/",",")
    tempstr=tempstr.replace("，",",")
    tempstr=tempstr.replace(" ",",")
    tempstr=tempstr.replace("／",",")
    tempstr=tempstr.replace(",,",",")
    tempstr=tempstr.replace("、",",")
    keywords[i]=tempstr.upper
    totalKeys+=tempstr.split(",")
#print(totalKeys)

#----- 分析關鍵字並提出建議
#{關鍵字:出現次數}建立字典並排序
import pandas as pd
sugKeys=dict()
for i in range(len(totalKeys)):
    sugKeys.update({totalKeys[i]:totalKeys.count(totalKeys[i])})
pdKey=pd.DataFrame(list(sugKeys.items()))
pdKey['sortNum']=pdKey.index
pdKey=pdKey.sort_values(by=[1,'sortNum'],ascending=(False,True))

print("相關聯關鍵字建議:")
for i in range(9):
    print(pdKey.iloc[i,0],end='、')
print(pdKey.iloc[9,0])

請輸入欲查詢的關鍵字:書桌
請輸入欲比對的公司網址或名稱:IKEA
Google搜尋資料擷取進度:  0%
Google搜尋資料擷取進度: 10%
Google搜尋資料擷取進度: 20%
Google搜尋資料擷取進度: 30%
Google搜尋資料擷取進度: 40%
Google搜尋資料擷取進度: 50%
Google搜尋資料擷取進度: 60%
Google搜尋資料擷取進度: 70%
Google搜尋資料擷取進度: 80%
Google搜尋資料擷取進度: 90%
Google搜尋資料擷取進度:100%
【IKEA】 在Google以 【書桌】關鍵字搜尋結果：
【IKEA書桌｜家用電腦書桌、商用升降工作桌、兒童書桌椅｜IKEA線上購物https://www.ikea.com.tw › products › work-desks】搜尋的名次為 3 名.
【網友激推IKEA書桌熱銷排行- 松果購物https://www.pcone.com.tw › search › q=IKEA書桌】搜尋的名次為 14 名.
關鍵字分析進度: 10%
關鍵字分析進度: 20%
關鍵字分析進度: 30%
關鍵字分析進度: 40%
關鍵字分析進度: 50%
關鍵字分析進度: 60%
關鍵字分析進度: 70%
關鍵字分析進度: 80%
關鍵字分析進度: 90%
關鍵字分析進度:100%
相關聯關鍵字建議:
電腦桌、書桌、小書桌、桌、桌子、設計家具、、團購、宅配、購物


In [2]:
pdKey

,0,1,sortNum
8,電腦桌,5,8
6,書桌,4,6
7,小書桌,2,7
9,桌,2,9
10,桌子,2,10
...,...,...,...
75,豌豆桌,1,75
76,辦公椅,1,76
77,寶寶小沙發,1,77
78,Bunny,1,78
